----
# Introduction to Data Science and Systems 2023-2024

## Challenge lab: Introduction to Jupyter, Python and Numpy [unassessed and ungraded]
University of Glasgow

v20232024a



---


### Notes
It is recommended to keep the lecture notes open while doing this lab exercise.

**This exercise is assessed**. Make sure you upload your solution by the deadline. See the notes at the bottom of this notebook for submission guidance.


### References
If you are stuck, the following resources are very helpful:


* [NumPy cheatsheet](https://github.com/juliangaal/python-cheat-sheet/blob/master/NumPy/NumPy.md)
* [NumPy API reference](https://docs.scipy.org/doc/numpy-1.13.0/reference/)
* [NumPy user guide](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.html)


* [Python for Data Science cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PythonForDataScience.pdf)
* [Another NumPy Cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)


## Purpose of this lab
This lab should help you:
* understand floating point representations
* understand how roundoff errors occur and how you can control them
* work with higher rank tensors, selecting the attributes you want to work with
* understand how to do simple operations in a vectorised manner

Note: this lab requires solving puzzles which require that you understand the course material. Very little code is required to get the correct solutions.


## Before you start...

In [ ]:
## custom utils
## uncomment and run the line below if you get an error about jhwutils
## then RESTART THE KERNEL (Kernel/Restart)

#!pip --no-cache install --user -U https://github.com/johnhw/jhwutils/zipball/master   

# comment it again before submitting!

Import some basic things

In [ ]:
# Standard imports
# Make sure you run this cell!
# NumPy
import numpy as np  
np.set_printoptions(suppress=True)

# Set up Matplotlib
import matplotlib as mpl   
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('figure', figsize=(8.0, 4.0), dpi=140)

from jhwutils.checkarr import array_hash, check_hash
from jhwutils.float_inspector import print_float_html
import jhwutils.image_audio as ia
import jhwutils.tick as tick

import lzma, base64
exec(lzma.decompress(base64.b64decode(b'/Td6WFoAAATm1rRGAgAhARYAAAB0L+Wj4AHWARFdAAUaCmNWCuiTCXe8bHWT/WeqghymfBRQKyklXJ3lgDWHk34myezvldkgSu3Adiur0vA+OkDfUwiMWzEclOxunCssCtgpVM94TwtylLQC9aX0APwnuNk2VBPkVpf3otXT04I1pElMWNdSgqgJ9/PqMJhdhfDr3Wrs/a/pRN/AOd+rZawioudIbGRTYZgWPHcqPLImmS2EO0Hbkc7kRAS3Nr9JkELrRMkejvVMnGgu+b1m4uXv6trDURkPrMO7HCVcO5FcMx1FURc+hNcKRmmBp1mCuW4iop6qRAMNnAur/spBmfuw+lbJkxOoIXMwrRuEXa6bnJz53WQnloXvzbWW5hqEtbPpSHPLPccxaiU5yPAKYAAAAADqkqJjsFbfFwABrQLXAwAA9LrSpbHEZ/sCAAAAAARZWg==')))

print("Everything imported OK")

## Precision
Want to change precsion of number printing in NumPy?

* `np.set_printoptions(precision=30)` will show the numbers in full
* `np.set_printoptions(precision=3)` will show three decimal places, etc.


# 1. Financial misconduct

You have been asked to verify the computation of some financial predictive models. These models produces a sequence of updates to the value of a product. The product updates are mainly of two types:
* **large deposits**, representing inflows of new cash, often up into the billions of pounds
* **small returns** from high-frequency trading activity

The simulator produces **two** model outputs from two distinct models `a` and `b` at each time step, which provide very similar estimates of the value of these updates.

You are asked to write code that will produce:

* an estimate of the total value of a product over some series
* the total difference between two different product models, both of which are very similar.

You are given the existing code below, which is supposed to compute and return:

* the sum of the `a` updates (i.e. total value of `a`)
* the accumulated difference between the `a` and `b` products.

However, the result is very inaccurate when tested. Modify this code to be more accurate. Do NOT use NumPy, or *any* other external module to improve your calculation. Use floating point, regardless of the fact that floating point is not appropriate for financial data.

The errors should be less than 0.5 for the `a` sum and less than 1e-10 for the difference in predictions.

In [ ]:
class Simulator: # we use a class just to hold variables between calls
    def __init__(self):
        # initialise accumulators
        self.a_sum = 0
        self.b_sum = 0
        
    def update(self, a, b):
        # increment
        self.a_sum += a
        self.b_sum += b
        
    def results(self):
        # return a  pair of results
        # (you do not need to change this)
        return self.a_sum, self.a_sum - self.b_sum
        

In [ ]:
a_error, d_error = simulate(Simulator())
# bad result!
print(f"Error in a_sum is {a_error} and {d_error} in d_sum")

Copy and paste the `Simulator` into the cell below and modify it:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
    
a_error, d_error = simulate(Simulator())
print(f"Error in a_sum is {a_error} and {d_error} in d_sum")

In [ ]:
with tick.marks(2):
    assert(a_error<2.0)

In [ ]:
with tick.marks(2):
    assert(a_error<0.5)

In [ ]:
with tick.marks(2):
    assert(d_error<1e-10)

In [ ]:
with tick.marks(2):
    assert(d_error<1e-12)

---

# 2. Debugging the dump

Scenario: In your first day in a new post in the IT team in a finance company, you are provided with the a portion of a memory dump of a process that was running an important simulation of foreign exchange rates in the late 1990s. Unfortunately, the system crashed half way through and the raw memory dump is all that is left. You need to extract the relevant data so that the simulation can be restarted. 

You know the data is stored as a numerical array, so it has some known structure. You don't know the dtype or shape of the array, or where it starts or ends in the memory dump, however.

**This is a puzzle which will require careful thinking, but very little code to be written**

In [ ]:
# read the data in
with open("data/crash_bytes.dump", "rb")  as f:
    crash_dump = f.read()

In [ ]:
# the raw memory dump, in hex. This isn't too useful...
# but align, group, offset might be helpful to tweak...
def print_hex(x, align=20, group=5, offset=0):
    for i, byte in enumerate(x[offset:]):                
        print("%02X" % byte, end="\n" if i%align==align-1 else "  " if i%group==group-1 else " ")            
    
print_hex(crash_dump)

### What you know
All you have is the block of raw data you can see above.  You know the array is in there, but not exactly where it starts or stops.  The header information is gone, so there is no striding information/dope vector to go by.

* You know that the first column of the array is an increasing index, starting at 1.0

         a        b      c    ...
         1.0     ... 
         2.0     
         3.0     
         4.0     
              
* You also know that all values are finite are smaller than 100 billion
* All values in the array are known to be positive.
* You can assume the data is some form of IEEE 754, though you do not know what specific type.
* The data starts on a byte boundary.

**This is sufficient information to solve the whole puzzle**

### Task
Recover the data, formatted correctly, and store it in the variable `recovered_array`. 

* This will take some trial and error (although there *is* a relatively fast way to do it).
      

* You can convert the data to a NumPy array like this:
`np.frombuffer(bytes, dtype, count, offset)`
* `bytes` the data to decode, as raw bytes
* `dtype` the datatype of the data to decode
* `count` the number of **elements** in the array
* `offset` **in bytes** to start recovering data
    

In [ ]:
# A wrong example:
# try and read 18 words from offset 0
# reshape to a 6,3 array
# this clearly isn't right, as you will see
np.frombuffer(crash_dump, dtype=np.float64, count=18, offset=0).reshape(6,3)

* A hint: you can show how any NumPy memory will appear in memory in hex using `tobytes()` -- see the example below. Also, remember you need to infer the *shape* of the array.

In [ ]:
# create a simple array, and then get the raw bytes and print them
print_hex(np.array([[1.0, 2.0, 3.0], 
                    [4.0, 0.0, 0.0]], dtype=np.float64).tobytes())

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# test the shape
with tick.marks(5):        
    assert(check_hash(recovered_array.shape, ((2,), 59.5)))

In [ ]:
# test if the result is correct   
with tick.marks(8):        
    assert(np.allclose(array_hash(recovered_array)[1], 411436.8382024765, atol=1e-2, rtol=1e-2))

---

# 3. Working with tensors
The file `data/font_sheet.png` contains a number of characters in different fonts. It is an image which consists of the images of each *printable* ASCII character, (characters 32-128) arranged left to right. Each character image is precisely square. 

These are the characters present, in order:

In [ ]:
chars = "".join([chr(i) for i in range(32,128)])
print(chars)

  
Each font is also stacked left to right, so the image is one *very* long strip of characters. The image is grayscale.

In [ ]:
import skimage
fname = "data/font_sheet.png"
img_read = skimage.io.imread(fname, plugin='pil')[:,:,:3]
img_gray = skimage.color.rgb2gray(img_read)
all_fonts = img_gray.astype(float)
print(all_fonts.shape)

In [ ]:
# show a portion of the image
ia.show_image(all_fonts[64:128, 1024:2048])

Task A. Rearrange the image into a tensor called `font_sheet` that is ordered like this:

        (font, character, rows, cols)
        
* Showing the image `font_sheet[16, 33, :, :]` should show the "A" character of the 17th font.
* Showing the image `font_sheet[10, 1, :, :]` should be the "!" character of the 11th font.

In [ ]:
## hint
from jhwutils.matrices import show_boxed_tensor_latex
n = np.arange(36).reshape(2*3, 3*2)
show_boxed_tensor_latex(n, box_rows=False)
show_boxed_tensor_latex(n.reshape(2,3,2,3), box_rows=False)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# if your code worked, you should see an ! below
ia.show_image_mpl(font_sheet[10, 1, :, :])

In [ ]:
# if your code worked, you should see a gif of letters below
ia.show_gif(font_sheet[8, 33:33+26, :, :], width="20%")


In [ ]:
# test shape is correct
with tick.marks(6):        
    assert(check_hash(font_sheet.shape, ((4,), 944.9499472573252)))

In [ ]:
# test content is ok
with tick.marks(10):    
    assert(np.allclose(array_hash(font_sheet)[1], 55441039333148.88, atol=1e-2, rtol=1e-2))    

B. Create an array `letter_sample`, which will be a 2D image containing one letter from each of the letters in the lowercase alphabet, with each character in a different font. The results should show "a" in font 0, "b" in font 1, "c" in font 2 and so on, as a single continuous strip.

The letters should be arranged horizontally and contiguously in a strip in the output image:

       abcdefghijklmnopqrstuvwxyz


Hint:
* you will have to partially *undo* some of the swapping/reshaping you did earlier to get the data in the right format
* you need to use fancy indexing
* you'll need to slice -- work out how to slice the array correctly
* do not use a loop

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# each character should appear in a different font
ia.show_image(letter_sample)

In [ ]:

with tick.marks(8):
    assert(check_hash(letter_sample,((64, 1664), 4815084617.957046)))

C. Compute the average representation of the letter "x" by taking the 64x64 mean image of the letter `x` across all fonts and store it in `average_x`.

        

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# show the result -- should be a 64x64 image
ia.show_image_mpl(average_x)

In [ ]:
with tick.marks(5):    
    assert(check_hash(average_x, ((64, 64), 7278599.401423973)))

----

## Rendering fonts
Complete the function below. It should render text using the provided font index, and *return* a single array with the text rendered in a horizontal strip. It should use `font_sheet` that you defined earlier. You can assume equal spacing of letters. 

* You can compute the index of the character in the same units as the font sheet using the formula:

      ix = ord(char) - 32
    
Every ASCII character (32-127) should be rendered. Any character that could not be rendered should be rendered as a **blank white** square.    

* It is fine to use a `for` loop to solve this problem

In [ ]:
def render_text(string, font_index):
    """Returns an image with the given string rendered, using the font_index selected.
    Reads characters from font_sheet.
    string: String to be rendered.
    font_index: index of the font to use"""
    pass # you can delete this line
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# you should be able to read this
ia.show_image(render_text("Can you see this clearly?", 23))

In [ ]:
# this should look the same
ia.show_image(render_text("Can\tyou\nsee\xf5this\x00clearly?", 23))

In [ ]:
ia.show_image(render_text("IDSS", 1))

In [ ]:
with tick.marks(0):
    assert(check_hash(render_text("Test 1", 1), ((64, 384), 269160963.20571893)))

In [ ]:
with tick.marks(0):
    assert(check_hash(render_text("Test 2", 2),((64, 384), 282670129.18082076)))

In [ ]:
with tick.marks(0):
    assert(check_hash(render_text("Test\n3", 3), ((64, 384), 283057779.18977338)))

In [ ]:
with tick.marks(0):
    assert(check_hash(render_text("\n\tTest\x00\xff4", 4), ((64, 576), 657469474.43368447)))

---

# Submission on Moodle

You are welcome to submit the lab on Moodle but we will not be grading it. There is no PDF generated for this challenge lab.